# Loading and Extracting data

In [1]:
# importing necessary packages


import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

In [3]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
        
# using an if here because tags can repeat    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

# Tokenizing input data and creating categorical targets (using label encoder)

In [4]:
# convert the target labels into a model understandable form.
# “LabelEncoder()” function provided by scikit-learn
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [6]:
vocab_size = 1000 # the number of words to keep, based on frequency
# why we limit vocab size?
# we don’t want every unique token in our vocabulary. If it doesn’t appear at 
# least twice then might just be a spelling mistake or a 
# word we can’t learn anything about it if it doesn’t appear that often.

# we might think that low frequency words (especially hapax legomena) 
# don't tell us very much useful information! And practically, this is often true; you don't 
# benefit from increasing the vocab size past a certain point, 
# but you will continue to incur increasing memory and performance costs associated with a larger input representation.

embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # oov_token is a value for “out of token”

# Tokenization is a method to segregate a particular text into small chunks or tokens. 
# Here the tokens or chunks can be anything from words to characters, even subwords.

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

# fitting the tokenizer on training sentences
tokenizer.fit_on_texts(training_sentences)

# the index of the words in tokenizer. this is a dict
word_index = tokenizer.word_index

# texts_to_sequences method in tokenizer class helps in converting tokens of text corpus into a sequence of integers.
sequences = tokenizer.texts_to_sequences(training_sentences)

# The “pad_sequences” method is used to make all the training text sequences into the same size.
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Building model


In [7]:
# model training using sequential model of keras

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [8]:
# fitting the model on the training labels and training sequences
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0770 - accuracy: 0.0303
Epoch 2/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0751 - accuracy: 0.2121
Epoch 3/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0743 - accuracy: 0.2121
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0735 - accuracy: 0.2121
Epoch 5/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0729 - accuracy: 0.1818
Epoch 6/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0727 - accuracy: 0.1212
Epoch 7/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0721 - accuracy: 0.1212
Epoch 8/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0713 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0708 - accuracy: 0.2121
Epoch 10/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0699 - accuracy: 0.2424
Epoch 11/500
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 1.9993 - accuracy: 0.2121
Epoch 85/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9979 - accuracy: 0.2121
Epoch 86/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9964 - accuracy: 0.2121
Epoch 87/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9951 - accuracy: 0.2121
Epoch 88/500
2/2 [==============================] - 0s 2ms/step - loss: 1.9939 - accuracy: 0.2121
Epoch 89/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9930 - accuracy: 0.2121
Epoch 90/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9921 - accuracy: 0.2121
Epoch 91/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9911 - accuracy: 0.2121
Epoch 92/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9900 - accuracy: 0.2121
Epoch 93/500
2/2 [==============================] - 0s 1ms/step - loss: 1.9893 - accuracy: 0.2121
Epoch 94/500
2/2 [===============

Epoch 167/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8363 - accuracy: 0.3333
Epoch 168/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8343 - accuracy: 0.3333
Epoch 169/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8324 - accuracy: 0.3333
Epoch 170/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8310 - accuracy: 0.3333
Epoch 171/500
2/2 [==============================] - 0s 1000us/step - loss: 1.8290 - accuracy: 0.3333
Epoch 172/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8273 - accuracy: 0.3333
Epoch 173/500
2/2 [==============================] - 0s 1000us/step - loss: 1.8247 - accuracy: 0.3333
Epoch 174/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8211 - accuracy: 0.3333
Epoch 175/500
2/2 [==============================] - 0s 1ms/step - loss: 1.8175 - accuracy: 0.3333
Epoch 176/500
2/2 [==============================] - 0s 2ms/step - loss: 1.8135 - accuracy: 0.3333
Epoc

2/2 [==============================] - 0s 1ms/step - loss: 1.5050 - accuracy: 0.4848
Epoch 250/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4981 - accuracy: 0.4848
Epoch 251/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4913 - accuracy: 0.4848
Epoch 252/500
2/2 [==============================] - 0s 3ms/step - loss: 1.4856 - accuracy: 0.5152
Epoch 253/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4797 - accuracy: 0.5758
Epoch 254/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4744 - accuracy: 0.5758
Epoch 255/500
2/2 [==============================] - 0s 1ms/step - loss: 1.4695 - accuracy: 0.5758
Epoch 256/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4644 - accuracy: 0.6061
Epoch 257/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4585 - accuracy: 0.6061
Epoch 258/500
2/2 [==============================] - 0s 2ms/step - loss: 1.4531 - accuracy: 0.6061
Epoch 259/500
2/2 [=====

2/2 [==============================] - 0s 3ms/step - loss: 1.0783 - accuracy: 0.6970
Epoch 332/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0752 - accuracy: 0.7273
Epoch 333/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0718 - accuracy: 0.7576
Epoch 334/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0680 - accuracy: 0.7576
Epoch 335/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0640 - accuracy: 0.7576
Epoch 336/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0601 - accuracy: 0.7576
Epoch 337/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0557 - accuracy: 0.7576
Epoch 338/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0511 - accuracy: 0.7576
Epoch 339/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0466 - accuracy: 0.7576
Epoch 340/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0421 - accuracy: 0.7576
Epoch 341/500
2/2 [=====

2/2 [==============================] - 0s 2ms/step - loss: 0.7936 - accuracy: 0.8182
Epoch 414/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7896 - accuracy: 0.8182
Epoch 415/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7848 - accuracy: 0.8182
Epoch 416/500
2/2 [==============================] - 0s 1ms/step - loss: 0.7807 - accuracy: 0.8182
Epoch 417/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7772 - accuracy: 0.8182
Epoch 418/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7737 - accuracy: 0.8182
Epoch 419/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7706 - accuracy: 0.8182
Epoch 420/500
2/2 [==============================] - 0s 2ms/step - loss: 0.7676 - accuracy: 0.8182
Epoch 421/500
2/2 [==============================] - 0s 1ms/step - loss: 0.7638 - accuracy: 0.8182
Epoch 422/500
2/2 [==============================] - 0s 3ms/step - loss: 0.7590 - accuracy: 0.8182
Epoch 423/500
2/2 [=====

2/2 [==============================] - 0s 2ms/step - loss: 0.5817 - accuracy: 0.8485
Epoch 496/500
2/2 [==============================] - 0s 1ms/step - loss: 0.5803 - accuracy: 0.8182
Epoch 497/500
2/2 [==============================] - 0s 2ms/step - loss: 0.5800 - accuracy: 0.8788
Epoch 498/500
2/2 [==============================] - 0s 2ms/step - loss: 0.5788 - accuracy: 0.8788
Epoch 499/500
2/2 [==============================] - 0s 2ms/step - loss: 0.5759 - accuracy: 0.8788
Epoch 500/500
2/2 [==============================] - 0s 2ms/step - loss: 0.5725 - accuracy: 0.8788


In [9]:
# save all the required files in order to use it at the inference time. 
# we save the trained model, fitted tokenizer object and fitted label encoder object.

# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


# The Chat Bot

In [10]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hello!
ChatBot: Hello
User: What is your name?
ChatBot: I'm Joana!
User: Tell me a joke?
ChatBot: Tell me how can assist you
User: How are you doing today?
ChatBot: I.m Joana, your bot assistant
User: quit
